In [1]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

C:\Users\ivolochkov\Documents\jupyter_projects\web\webtrade\template_stocks1.csv


In [ ]:
#monkey.patch_all(thread=False,select=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

127.0.0.1 - - [2021-07-22 10:59:50] "GET / HTTP/1.1" 200 8724 0.020630
127.0.0.1 - - [2021-07-22 10:59:50] "POST /query_data?view_id=open_broker_report&params=_173364_ HTTP/1.1" 200 8489 0.026365
127.0.0.1 - - [2021-07-22 10:59:50] "POST /query_data?view_id=open_broker_report&params=_173364i_ HTTP/1.1" 200 992 0.008880
127.0.0.1 - - [2021-07-22 10:59:50] "POST /query_data?view_id=open_broker_report&params=_173364_ HTTP/1.1" 200 8489 0.022310
127.0.0.1 - - [2021-07-22 10:59:50] "POST /query_data?view_id=open_broker_report&params=_173364i_ HTTP/1.1" 200 992 0.007001
127.0.0.1 - - [2021-07-22 10:59:50] "POST /query_data?table=sec_sector&noconvert=1 HTTP/1.1" 200 4031 0.015038
127.0.0.1 - - [2021-07-22 10:59:51] "GET /portfolio_stocks HTTP/1.1" 200 6959 0.006604
127.0.0.1 - - [2021-07-22 10:59:53] "GET /portfolio_stocks HTTP/1.1" 200 6959 0.001048
127.0.0.1 - - [2021-07-22 10:59:53] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57471 0.007491
127.0.0.1 - - [2021-07-22 

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
%%time
from sys import path
from datetime import datetime
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
import pandas as pd
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

In [ ]:
a=job_sectors(init_db_manager())

In [ ]:
db=init_db_manager()
a=db.get_table('mosex_securities',result='matrix')
#a=job_update_sec_hist(init_db_manager())

In [ ]:
def __prepare_csv_f(s):
    if isnan(s):
        return ""
    return str(s)

In [ ]:
#pd.DataFrame(a[1:],columns=a[0]).to_csv("to_csv.csv",index=False,encoding='cp1251')
"aaaa".encode("cp1251")

In [ ]:
for row in db.get_table_cursor('sec_sector',query={'ticker': ['IRKT','RKKE']}):
    print(list(row.values()))

In [ ]:
for row in db.db['sec_sector'].find({'ticker':{'$in': ['IRKT','RKKE']}}):
    print(row)

In [ ]:
db.get_table('sec_sector',query={'ticker':{'$in': ['IRKT','RKKE']}},dict_key="ticker",result='dict',columns=['action',
                                                                                                             'industry',
                                                                                                             'sector','ticker'])


In [ ]:
def f(c=11,a=3,b=6):
    return a/b+c
aa={'a':5,'b':15}
for k,v in aa.items():
    print(k,v)

In [ ]:
list(row.values())

In [ ]:
db.find_one('sec_sector',{'ticker': ['IRKT']})

In [ ]:
a=pd.DataFrame(a)
a

In [ ]:
a.shape

In [ ]:
db.get_table('sec_history_manager_mosex')

In [ ]:
db.get_table('mosex_sec_history')[-10:]

In [ ]:
r=db.find_one('sec_history_manager_mosex',{"secid":'GAZP111'})

In [ ]:
from datetime import datetime
datetime.strptime('2016-01-04','%Y-%m-%d').month

In [ ]:
db=init_db_manager()

In [ ]:
df[df.boardid=='TQBR'].tradedate.nunique()

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
#a=tradingview()

In [ ]:

aa=a.security_hist('SBER')

In [ ]:
aa[1][1]